In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/ES/"

In [3]:
files = sorted(os.listdir(folder))
len(files)

1

In [4]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

50

In [5]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(50, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,Dilujc delVeneralleYarcn pues de a ver repetid...,@@@@@@@@@@Dilujc del@Veneralle@Yarcn@@@ pues d...,cuarenta Dibujo del Venerable Varon 42 pues d...
1,"M E D tir a Juan Bentiyollu, que con Ginebra S...","M E D tir a Juan Bentiyollu, que con Ginebra S...","@ MED tir à Juan Bentivollo, que con Ginebra S..."
2,"CAN no tienen oficio,ni domicílio.LatAa/afro,!...",@CA@@N@@@@@ no@@@@@ @@@ti@@@@@@@@@@@@@en@@@@@@...,CAN Ne ffs sodalis rustico: cave prudens. CAñ...
3,"[ ' il 1 592 M O L MOLINILLO, f. m. Dim. El in...","[ ' il 1 592 M O L MOLINILLO, f. m. Dim. El in...",@@@@@@@@ 592@@@@@@ MOLINILLO. s. m. Dim. El in...
4,C 30 Declaración de ¡4 don que en ellas haze d...,C 30 Declaración de ¡4 @don que en ellas haze ...,@@@@ Declaracion de la cion que en ellas haze ...


In [7]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,50.000000,50.000000,50.000000
mean,2696.200000,2986.560000,2985.560000
std,1862.357865,2171.228742,2171.228742
min,276.000000,308.000000,307.000000
25%,1242.000000,1287.750000,1286.750000
50%,1795.000000,1922.500000,1921.500000
75%,5244.500000,5649.500000,5648.500000
max,5878.000000,7367.000000,7366.000000


In [8]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

161


In [9]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count       50.000000
mean       333.967684
std       1600.057288
min          6.842349
25%         11.825675
50%         28.188309
75%         60.863019
max      10600.000000
Name: cer, dtype: float64

In [10]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    50.000000
mean     33.536902
std      25.130719
min       6.795225
25%      11.452404
50%      25.440666
75%      54.758002
max      99.089791
Name: cer, dtype: float64

In [11]:
data.to_pickle("../../data/es/data/test.pkl")